In [ ]:
import os
import sys
import shutil

from google.colab import drive
drive.mount('/content/drive')

#@title ## 🚩 Start Here

#@markdown ### ▶️ 設定
#@markdown 雲端硬碟中圖片資料夾名稱，通常為 10_images 這樣的名稱。<p>例如：你的雲端硬碟中，圖片放在 `MyDrive/Caption/9_girlimage`，則這裡請填入 `Caption/9_girlimage`。
image_folder = "SDModels/9_hinaigirl_alita" #@param {type:"string"}
#@markdown 使用前置角色名參數，自動由圖片資料夾中 _ 後方的名字為角色名稱
folder_name = True #@param {type:"boolean"}
#@markdown 不使用前置角色名，勾選此項目會自動忽略 `folder_name` 設定
not_char = False #@param {type:"boolean"}
#@markdown 使用前置服裝標籤（單人）
clothtag = True #@param {type:"boolean"}
#@markdown 前置多人動作標（當有雙人時）
peopletag = False #@param {type:"boolean"}
#@markdown 刪除 WD14 所產生的顏色標（前置與 Florence2 的會保留）
drop_colortag = True #@param {type:"boolean"}
#@markdown 從 n 天內打的標繼續
continue_caption = 0 #@param {type:"integer"}
#@markdown 只打一行 也許正則圖這樣做比較好（不確定沒測試）
rawdata = False #@param {type:"boolean"}
#@markdown 升級腳本，有需要才用
upgrade = False #@param {type:"boolean"}

#@markdown ### ▶️ 準備好了
#@markdown 你現在可以執行此儲存格來開始標記圖片，祝你好運！

# 👩‍💻 Cool code goes here
root_dir       = "/content"
main_dir       = os.path.join(root_dir, "drive/MyDrive")
dataset_folder = os.path.join(main_dir, image_folder)

def download_file(url, filename):
  import requests
  try:
    response = requests.get(url)
    response.raise_for_status()
    with open(filename, 'wb') as file:
      file.write(response.content)
    print(f"Downloaded {filename} from {url}")
  except requests.exceptions.RequestException as e:
    print(f"Failed to download {filename} from {url}: {e}")
    sys.exit(1)

def install_dependencies():
  os.chdir(root_dir)

  checkModule = !python -c 'import pkgutil; print(1 if pkgutil.find_loader("onnxruntime") else 0)'
  checkModule = int(checkModule[0]) > 0
  if not checkModule:
    !pip install argparse datetime onnxruntime-gpu==1.18.1 ftfy dghs-imgutils[gpu] timm aesthetic-predictor-v2-5

  checkModule = !python -c 'import pkgutil; print(1 if pkgutil.find_loader("flash_attn") else 0)'
  checkModule = int(checkModule[0]) > 0
  if not checkModule:
    !pip install flash_attn==2.5.9.post1

  !pip cache purge

  if not os.path.exists('./checkpoints'):
    if not os.path.exists('Long-CLIP'):
      !git clone https://github.com/beichenzbc/Long-CLIP

    for filename in os.listdir('Long-CLIP'):
      shutil.move(os.path.join('Long-CLIP', filename), '.')

    shutil.rmtree('Long-CLIP')

  weight_url = 'https://huggingface.co/zer0int/LongCLIP-GmP-ViT-L-14/resolve/main/Long-ViT-L-14-GmP-ft-state_dict.pt?download=true'
  weight_path = './checkpoints/Long-ViT-L-14-GmP-ft-state_dict.pt'
  if not os.path.exists(weight_path):
    download_file(weight_url, weight_path)

def run_main_script_in_venv():
  main_script_url = "https://raw.githubusercontent.com/gesen2egee/dataset_tools/main/main_script.py"
  main_script_filename = "main_script.py"

  if not os.path.exists(main_script_filename) or True == upgrade:
    download_file(main_script_url, main_script_filename)

  command = f"{main_script_filename} '{dataset_folder}'"

  if folder_name and not not_char:
    command = command + " --folder_name"
  else:
    if not_char:
      command = command + " --not_char"
  if clothtag:
    command = command + " --clothtag"
  if peopletag:
    command = command + " --peopletag"
  if drop_colortag:
    command = command + " --drop_colortag"
  if continue_caption > 0:
    command = command + f" --continue_caption {continue_caption}"
  if rawdata:
    command = command + " --rawdata"

  print(f"Execute command: {command}")

  !python {command}

def main():
  install_dependencies()
  run_main_script_in_venv()

main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files removed: 0
Execute command: main_script.py '/content/drive/MyDrive/SDModels/9_hinaigirl_alita' --folder_name --clothtag --drop_colortag
2024-07-04 06:19:16.040425: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 06:19:16.040486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 06:19:16.042122: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 06:19:16.053439: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl